In [46]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pytfa

from optlang.exceptions import SolverError

#from cobra.core.model import SolverNotFound
from cobra.flux_analysis import flux_variability_analysis
from cobra.io import load_matlab_model, load_json_model


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data
from pytfa.optim.relaxation import relax_dgo

CPLEX = 'optlang-cplex'
GUROBI = 'optlang-gurobi'
GLPK = 'optlang-glpk'
solver = GLPK

case = 'full' # 'reduced' or full'
pytfba_path = '/Users/joshuagoldford/Documents/github/pytfa'



# Load reaction DB
print("Loading thermo data...")

thermo_data = load_thermoDB(pytfba_path + '/data/thermo_data.thermodb')

print("Done !")

if case == 'reduced':
    cobra_model = import_matlab_model(pytfba_path + '/models/small_ecoli.mat')
    mytfa = pytfa.ThermoModel(thermo_data, cobra_model)
    biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'
elif case == 'full':
    biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'

    # We import pre-compiled data as it is faster for bigger models
    cobra_model = load_json_model(pytfba_path + '/models/iJO1366.json')

    # set the model constraints
    #cbmodel.objective = {cbmodel.reactions.get_by_id(biomass_rxn): 1}
    flux_value = 0.1
    cobra_model.reactions.get_by_id(biomass_rxn).lower_bound = flux_value
    cobra_model.reactions.get_by_id(biomass_rxn).upper_bound = flux_value

    lexicon = read_lexicon(pytfba_path + '/models/iJO1366/lexicon.csv')
    compartment_data = read_compartment_data(pytfba_path + '/models/iJO1366/compartment_data.json')

    # Initialize the cobra_model
    mytfa = pytfa.ThermoModel(thermo_data, cobra_model)

    # Annotate the cobra_model
    annotate_from_lexicon(mytfa, lexicon)
    apply_compartment_data(mytfa, compartment_data)

    biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'

mytfa.name = 'tutorial_basics'
mytfa.solver = solver
mytfa.objective = biomass_rxn

# Solver settings

def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)


## FBA
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value
# fva = flux_variability_analysis(mytfa)



Loading thermo data...
Done !


2021-08-24 16:35:33,771 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [48]:
fba_solution

,fluxes,reduced_costs
DM_4CRSOL,2.230000e-05,0.0
DM_5DRIB,2.310000e-05,0.0
DM_AACALD,0.000000e+00,0.0
DM_AMOB,2.000000e-07,0.0
DM_MTHTHF,1.340000e-04,0.0
...,...,...
ZN2abcpp,0.000000e+00,0.0
ZN2t3pp,0.000000e+00,0.0
ZN2tpp,3.240000e-05,0.0
ZNabcpp,0.000000e+00,0.0


In [ ]:

## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

## Info on the cobra_model
mytfa.print_info()

## Optimality
tfa_solution = mytfa.optimize()
tfa_value = tfa_solution.objective_value

# It might happen that the model is infeasible. In this case, we can relax 
# thermodynamics constraints:

if tfa_value < 0.1:
    from pytfa.optim.relaxation import relax_dgo

    mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5*fba_value
    relaxed_model, slack_model, relax_table = relax_dgo(mytfa)

    original_model, mytfa = mytfa, relaxed_model

    print('Relaxation: ')
    print(relax_table)
    
    tfa_solution = mytfa.optimize()
    tfa_value = tfa_solution.objective_value

# Report
print('FBA Solution found : {0:.5g}'.format(fba_value))
print('TFA Solution found : {0:.5g}'.format(tfa_value))


solver_results = dict()

# Try different solvers

2021-08-24 16:35:59,793 - thermomodel_None - INFO - # Model preparation starting...
2021-08-24 16:36:02,711 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-24 16:36:02,712 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:02,713 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:02,714 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-24 16:36:02,715 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:02,716 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:03,034 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-24 16:36:03,034 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:03,035 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:03,035 - thermomodel_None - WARNING - Warning : NULL/U
2021-08-24 16:36:03,036 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:03,037 - thermomodel_None - WARNING - Warning : NULL/L
2021-08-24 16:36:03,240 - thermomodel_None - WARNING

                           value
key                             
name             tutorial_basics
description      tutorial_basics
num constraints            15034
num variables              15538
num metabolites             1807
num reactions               2585
                             value
key                               
num metabolites(thermo)       1550
num reactions(thermo)         1824
pct metabolites(thermo)  85.777532
pct reactions(thermo)    70.560928


/Users/joshuagoldford/opt/miniconda3/envs/coenzymes/lib/python3.8/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
2021-08-24 16:36:29,098 - thermomodel_tutorial_basics - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-08-24 16:36:39,785 - thermomodel_tutorial_basics - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2021-08-24 16:36:46,732 - thermomodel_tutorial_basics - INFO - Adding slack constraints
adding slacks: 100%|██████████| 1824/1824 [00:11<00:00, 152.58it/s]
